<div>
    <table>
        <tr>
            <td>
                <center>
                    <h1>Brightway (2.5) Introduction</h1>
                     <a href="https://www.psi.ch/en/ta/people/romain-sacchi">Romain Sacchi</a> (PSI)
                     <a href="https://www.psi.ch/en/ta/people/karin-treyer">Karin Treyer</a> (PSI)
                    <br><br>
                    Duration: 1 hour 15 minutes.
                </center>
            </td>
        </tr>
    </div>

# Brightway modules: bw2io, bw2data, bw2calc

*The following content is adapted from [the course materials dispensed at the 2022 Brightcon conference](https://github.com/Depart-de-Sentier/teaching-material/tree/main/beginners/Projects%2C%20databases%2C%20exchanges%2C%20activities). If you need additional notebooks, go to this repository*.


<div class="alert alert-info">
Note: we will be using <a href="https://docs.brightway.dev/en/latest/content/installation/index.html">Brightway 2.5</a>, not <a href="https://docs.brightway.dev/en/legacy/">Brightway 2</a>. From the user end side, very little differ between the two. The code executed throughout this notebook works with both versions... probably.
</div>





**Brightway2 documentation** 👉 [https://docs.brightway.dev/en/legacy/index.html](https://docs.brightway.dev/en/legacy/index.html)

**Brightway25 documentation** 👉 [https://docs.brightway.dev/en/latest/index.html](https://docs.brightway.dev/en/latest/index.html)

### Learning objectives  
  - Learn about the general structure of Brightway and its most importand abstractions: projects, databases, activities and exchanges  
  - Learn how to find objects (notably activities and exchanges), assign them to variables and work with them using their associated methods  
  - Learn about how simple LCA calculations are done (one product, one impact category), and specifically how the different matrices are built and used  
  - Learn how to extract information from the matrices (inputs or results) and translate them into nice, human-readable objects  
  - Learn different ways to carry out comparative LCAs  
  - Learn different ways to carry out LCAs with multiple impact categories

### Content  

#### 1) Getting started  
##### 1.1) Brightway
  - Accessing Brightway libraries  
  
##### 1.2) Projects  
  - Concept  
  - Creating a new project, or switching to an existing project  
  - Contents of a project  
  
##### 1.3) bw2_setup(), biosphere3 database and LCIA methods  
  - bw2_setup()  
  - biosphere3 database and a first look at database objects  
  - Getting activities from codes or keys  
  - Methods  
  - Looking up elementary flows (list comprehensions, search)  
  - Searching for methods  
  - Nice display of data in methods 

##### 1.4) LCI databases  
  - Importing (succinct)  
  - LCI activities  
  - Looking up activities  
  - LCI exchanges
  - Loaded LCI databases
  
#### 2) My first LCA - simplest case:  
##### 2.1) General syntax of LCA calculations  

##### 2.2) The `demand` attribute  

##### 2.3) Reminder of the system that needs to be solved in calculating an LCI  

##### 2.4) Building the matrices  

  - $A$ matrix  
  - $B$ matrix  
  - $f$ (demand array)  
  
##### 2.5) Solution to the inventory calculation  

  - Supply array  
  - Inventory matrix  
  
##### 2.6) Life Cycle Impact Assessment  

  - `.lcia()` method  
  - Simple contribution analysis  
  
#### 3) My second LCA - comparative LCA:
    
#### 4) My third LCA - Multiple impact categories
  
#### 5) My first and third LCAs revisited with MultiLCA

### 1) Getting started

#### 1.1) Jupyter lab / notebooks and accessing Brightway2

This teaching material was produced with the intention to be used in Jupyter Notebooks.

<div class="alert alert-info">
This notebook is meant to run with a specific conda (and python) environment.
    Make sure you have started Jupyter Notebooks or Jupyter Lab from the conda
    environment where the brightway modules are installed (the conda environment called `bw25`).
</div>

##### Accessing Brightway libraries

The different modules in Brightway are Python libraries. This means that, to use them, you can use any environment from which you usually use Python (Idle, command prompt, Spyder or, as is the case today, Jupyter Notebooks).  

We will favour Jupyter Notebooks in this course because they allow us to integrate code and text and provide code snippets for you to complete.  

Like all other Python packages, you need to `import` Brightway modules.

Brightway is made up of *five* main modules:
* **bw2io**: to handle data input and output.
* **bw2data**: to handle databases and LCIA methods.
* **bw2calc**: to perform single-, multi- and Monte Carlo analyses.
* **bw2analyzer**: to analyze results produced by **bw2calc**.
* **bw2parameters**: to use parameters in inventories.

And many other modules for more specific uses. For example:
* **bw_processing**: to provide matrices to **bw2calc** directly, instead of a database.

We will for now import **bw2io**, **bw2data** and **bw2calc**.

In [ ]:
import bw2io, bw2data, bw2calc

We can check the versions of these libraries, and where their code is located on your computer/the server.

In [ ]:
bw2io.__version__

In [ ]:
bw2io.__file__

We're also going to be using the following libraries:

In [ ]:
import os               # to use "operating system dependent functionality"
import numpy as np      # "the fundamental package for scientific computing with Python"
import pandas as pd     # "high-performance, easy-to-use data structures and data analysis tools" for Python

#### 1.2) Projects

##### Concept

The top-level "container" in Brightway is the project (see [here](https://docs.brightway.dev/en/latest/content/theory/structure.html)). 

A project contains LCI databases, LCIA methods and other less often-used objects. Objects from one project do not interact with objects within other projects. By analogy, projects are like databases in other LCA software.  

![title](bw_structure.png)


When you first launch Brightway, you will be in the `default` project. You can check this using the `current` property of the `projects` object: 

In [ ]:
bw2data.projects.current

##### Creating a new project, or switching to an existing project

Let's create a new project for this course. Let's call it **"bw25_intro"**. 

There are two ways of doing this:  
* `projects.create_project('bw25_intro')` will create the project, but you will remain in your current project.
* `projects.set_current('bw25_intro')` will switch you to the project passed as an argument and create it first if it doesn't exist.  Let's do the latter:

In [ ]:
# The name of the project is entered as string; 
# it doesn't really have any restrictions, so can include spaces, 
# special characters, other languages, or even emoji.

bw2data.projects.set_current("bw25_intro")

You can always see what projects you have on your computer by running `list(bw.projects)`. Unless you have worked with Brightway2 before on your computer, your list should contain two projects: 'default' and 'bw2_seminar_2017'.

_**Exercise**_: list the projects on your computer.

_**Exercise**_: check the currently activated project.

Like in all Python modules, you can get additional information on the `projects` object and associated properties and methods by typing `help(projects)`. The [docs](https://docs.brightway.dev/en/latest/content/gettingstarted/projects.html) give you additional examples of the methods available from the project object.

##### Contents of a project

One property of `projects` is its location, given by `projects.dir`:

In [ ]:
bw2data.projects.dir

<div class="alert alert-info">
    If working on your personal computer, try to locate your project directory on your computer.
</div>

First things first: **do not panic**! You can use Brightway for years without ever opening this directory, but we will discuss some of these files later.

It is simply interesting to note that, for now, all the directories are empty except the `lci` directory, which contains an empty database.

All in all, our (empty) project takes less than 100KB.  
It is now time to start populating the project.

#### 1.3) bw2setup(), biosphere3 database and LCIA methods

Our project is empty, as it contains zero databases:

In [ ]:
bw2data.databases

##### The use of `bw2io.import_ecoinvent_release()` **when working on your computer**

`bw2io.import_ecoinvent_release()` is a convenient function to install three databases at once: 1) a **biosphere** database (elementary flows), 2) the **LCIA methods** database and 3) the **ecoinvent** database.

In [ ]:
bw2io.import_ecoinvent_release(
    version="3.10", 
    system_model="cutoff", # other options are "consequential", "apos" and "EN15804"
    username="xxxx",
    password="xxxx",
    biosphere_name="biosphere" # optional, otherwise a name is chosen for you
)

An alternative option is to use `bw2io.bw2setup()` which will install the `biosphere` and `LCIA methods` databases. It should be followed by `bw2io.SingleOutputEcospold2Importer()` to install ecoinvent from local EcoSpold2 files.

In [ ]:
bw2data.databases

The output tells us that import_ecoinvent_release() created some very useful things:  
  - Created a database called "biosphere": this database contains elementary flows (called biosphere exchanges in Brightway2)  
  - **762** impact assessment methods
  - the ecoinvent 3.10 cut-off database

<div class="alert alert-info">
    Note: The biosphere database contains all "natural" flows human-made activities connect to. E.g., CO$_{2}$ emissions, ore in ground, etc.
</div>
  
It also created some `mapping` between the imported exchanges and some integers: more on this later.  
The whole directory now takes up 1'640MB.

<div class="alert alert-info">
    Note: We have just created the biosphere database from a local file. <a href="https://files.brightway.dev">files.brightway.dev</a> also offers a few different online versions of the biosphere database. You can list them doing: bw2io.remote.PROJECTS_BW25
</div>

##### The *biosphere3* database and a first look at database objects

<div class="alert alert-info">
    Note: You can always list the databases inside a project by simply typing 'bw2data.databases'. This accesses the 'database.json' file in your 'project.dir' (I learned the latter by typing `bw2data.databases?`, you should try it too.).
</div>

In [ ]:
bw2data.databases

While not impossible to interact with the data at this level, you probably never will unless you are developping some funky program. Instead, it is strongly recommended to learn to work with `abstractions`.  

To access a database in Brightway, you use the `Database` initialization method (again, you can type `bw2data.Database?` for more information - this is the last time I'll mention this. The [docs](https://docs.brightway.dev/en/latest/content/gettingstarted/databases.html) also have an excellent page about the methods available from the Database object.

In [ ]:
bw2data.Database('biosphere')

It doesn't actually return anything other than information about the Backend.  
However, there are many properties and functions associated with this database object.  These are found [here](https://2.docs.brightway.dev/technical/bw2data.html#databasechooser). We can also have a look through the autocomplete. Let's assign the database to a variable:

In [ ]:
my_bio = bw2data.Database('biosphere')

Let's check the my_bio `type`:

In [ ]:
type(my_bio)

Let's check its length:

In [ ]:
len(my_bio)

This is exactly the number of items we saw had been added to databases.db  
Given this, what do you think is going on?

If you type `my_bio.` and click on tab, you should get a list of properties and methods associated with database objects. Try this now:

In [ ]:
my_bio.        # Type my_bio. and click tab. Have a look at the different properties and objects

Some of the more basic ones we will be using now are :  
  - `random()` - returns a random activity in the database
  - `get(*valid_exchange_tuple*)` - returns an activity, but you must know the activity key
  - `load()` - loads the whole database as a dictionary.
  - `make_searchable` - allows searching of the database (by default, it is already searchable)
  - `search` - search the database  
  
Lets start with `random`:

This returns a biosphere activity, but without assigning it to a variable, there is not much we can do with it directly.  

Note: It may seem counter-intuitive for elementary *flows* to be considered *activities* in Brightway, but it is no mistake. 
LCA models are made up of **nodes** (activities) that are linked by **edges** (exchanges). The biosphere activities are the nodes *outside* the technosphere. Emissions and resource extractions are modelled as exchanges between activities in the technosphere (part of the product system) and these biosphere activities.  

More on this later. 

For now, let's assign another random activity to a variable:

In [ ]:
random_biosphere = my_bio.random()
random_biosphere

We can get the type of the object that was returned from the database:

In [ ]:
type(random_biosphere)

The type is an **activity proxy**. Activity proxies allow us to interact with the content of the database.

In Brightway, we *almost* always work with `Activity` or `Exchange` objects. 

To see what the activity contains, we can convert it to a dictionary:

In [ ]:
random_biosphere.as_dict()

##### Getting activities from codes or keys

We can see that the activities in the biosphere3 database have unique **codes**, which we can use with the `get` function:

Activities can also be "gotten" via `get_activity`, but the argument is the activity **key**, consisting of a tuple with two elements: the database name, and the activity code.

**Exercise:** Use `bw.get_activity()` to retrieve the random biosphere activity. 

In [ ]:
database_name = 'biosphere'
code = random_biosphere['code']
random_biosphere_key = (database_name, code)
random_biosphere_key

In [ ]:
bw2data.get_activity(random_biosphere_key)

You can always find (or return) the key to an activity using the `.key` property.

In [ ]:
random_biosphere.key

##### Searching for activities

Let's say we are looking for a specific elementary flow, we can use the `search` method of the database (see [here](https://docs.brightwaylca.org/technical/bw2data.html#default-backend-databases-stored-in-a-sqlite-database) for more details on using search):

In [ ]:
bw2data.Database('biosphere').search('carbon dioxide')

It is also possible to use "filters" to narrow searches, e.g.

In [ ]:
bw2data.Database('biosphere').search('carbon dioxide', filter={'categories':'urban', 'name':'fossil'})

The database object is also iterable, allowing "home-made" searches through list comprehensions. This approach is better because one can add as many criteria as wanted:

In [ ]:
l = [
    act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
]

In [ ]:
type(l)

In [ ]:
len(l)

Python using a 0-based indexing, to fetch the first item of a list:

In [ ]:
l[0]

Activities returned by searches or list comprehensions can be assigned to variables, but to do so, one needs to identify the activity by index. Based on the above, I can refine my filters to ensure the list comprehension only returns one activity, and then choose it without fear of choosing the wrong one..

In [ ]:
[act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])]

In [ ]:
activity_I_want = [act for act in my_bio if 'Carbon dioxide' in act['name'] 
                                            and 'fossil' in act['name']
                                            and 'non' not in act['name']
                                            and 'urban air close to ground' in str(act['categories'])
         ][0]
activity_I_want

**Exercise**: look for and assign to a variable an emission of nitrous oxide emitted to air in the "urban air" subcompartment.

In [ ]:
# First inquiry:
[act for act in my_bio if 'nitrogen' in act['name']
                       and 'urban air' in str(act['categories'])
         ]

In [ ]:
# Found what I need:
[act for act in my_bio if 'Dinitrogen monoxide' in act['name']
                       and 'urban air close to ground' in str(act['categories'])
         ][0]

Let's leave the biosphere database here for now.

##### Methods

bw2_setup() also installed LCIA methods.

In [ ]:
bw2data.methods

One can load a random method:

In [ ]:
bw2data.methods.random()

In [ ]:
type(bw2data.methods.random())

Here, the random method returns the tuple by which the method is identified. To get to an actual method, the following syntax is used:

In [ ]:
bw2data.Method(('IPCC 2021', 'climate change', 'global warming potential (GWP100)'),)

Of course, a random method is probably not useful except to play around. To find an actual method, one can again use list comprehensions. Let's say I am interested in using the IPCC 2021 100 years method:

In [ ]:
[m for m in bw2data.methods if 'IPCC' in str(m) and ('2021') in str(m) and '100' in str(m) and "LT" not in str(m)]

I am interested in the second of these, and will assign it to a variable. I can will refine my search until there is one element in my list and then choose it by subscripting.

In [ ]:
[m for m in bw2data.methods if 'IPCC' in m[0]
                        and ('2021') in str(m)
                        and 'GWP100' in str(m)
                        and 'no LT' not in str(m)
                        and 'SLCF' not in str(m)
]

In [ ]:
# Good, now let's choose it:
ipcc2021 = [m for m in bw2data.methods if 'IPCC' in m[0]
                    and ('2021') in str(m)
                    and 'GWP100' in str(m)
                    and 'no LT' not in str(m)
                    and 'SLCF' not in str(m)
           ][0]

Of course, if I know exactly the method I want, and I know the syntax, I can simply type it out: `('IPCC 2021', 'climate change', 'GWP 100a')`

In [ ]:
type(ipcc2021)

In [ ]:
ipcc_2021_method = bw2data.Method(ipcc2021)

In [ ]:
type(ipcc_2021_method)

Again, there are a bunch of methods associated with a method object. You can access these by typing ipcc_2013_method. and clicking tab.  
For example, metadata:

In [ ]:
ipcc_2021_method.name

In [ ]:
ipcc_2021_method.metadata

In [ ]:
ipcc_2021_method.metadata['unit']

Question: where is this metadata?

Let's use the `load` method to see what is in the object:

In [ ]:
ipcc_2021_method.load()

This contains tuples with (elementary flow keys, characterization factors).

##### Nice display of data in methods 

**Exercise:** Create a dictionary with keys = elementary flow names and values = characterization factors for the `IPCC GWP100` method.

Bonus (optional): Generate a Pandas DataFrame with the resulting dictionary. 

<div class="alert alert-info">
    Note: The <a href="https://docs.brightway.dev/en/latest/content/gettingstarted/methods.html">docs</a> 
    give good examples on how to use `Method()`, including how to create your own LCIA method.
</div>

Enough said for now about methods.

#### 1.4) LCI databases

Other code to import LCI databases in other formats are found [here](https://github.com/brightway-lca/brightway2-io/tree/master/bw2io/importers). We'll explore this later.

*We created* a new project, so let's make sure we set it as the "current" project.

In [ ]:
bw2data.databases

To access the actual database, you need to use the Database method: 

Let's assign the database to a variable and see what we can do:

In [ ]:
eidb = bw2data.Database('ecoinvent-3.10-cutoff')

In [ ]:
# Check the length of the database:
len(eidb)

Again, we can get an idea of useful methods and attributes by typing eidb. and Tab. Do this now.

In [ ]:
eidb. #Press tab!

##### LCI activities

In the context of LCI databases, activities are the nodes "within the technosphere". They are therefore the columns in the technosphere matrix $A$.  
There are different ways to get access to an activity. Let's use the `random()` method for now to explore a random activity in the ecoinvent database.

In [ ]:
random_act = eidb.random()

In [ ]:
random_act

In [ ]:
type(random_act)

To see what is stored in an activity object, let's convert our random act in a dictionary: 

In [ ]:
from pprint import pprint
pprint(random_act.as_dict())

In [ ]:
random_act = eidb.random()
random_act

We can iterate through the exchanges and print them.

In [ ]:
for exc in list(random_act.exchanges()):
    print(exc.input["name"]," | ", exc.get("location"), " | ", exc["unit"] ," | ", exc["amount"]," | ", exc["type"])

##### Searching and getting LCI activities

Searching and getting LCI activities is done exactly the same way as for activities in the biosphere3 database:

In [ ]:
# Using search
eidb.search('transport', filter={'name':'lorry'})

In [ ]:
# Using list comprehensions:
[act for act in eidb if 'lorry' in act['name']
                    and 'RER' in act['location']
                    and '32' in act['name']
]

**Exercise:** Return an activity for electricity production, coal-fired power plants in Germany

In [ ]:
coal_DE = [act for act in eidb if 'electricity' in act['name']
                        and 'coal' in act['name']
                        and act['location']=='DE'
              ][0]

Let's print its exchanges in a pandas' DataFrame.

In [ ]:
pd.DataFrame(
    [
        [
            e["name"],
            e.get("location"),
            e["unit"],
            e["amount"],
            e["type"],
        ]
        for e in coal_DE.exchanges()
    ],
    columns=["Name", "Location", "Unit", "Amount", "Type"]
)

# note that we fetch "location" using .get(). Why?

#### LCI exchanges

**`Exchanges`** are the edges between nodes.

These can be:  
  - an edge between two activities within the technosphere (an element $a_{ij}$ of matrix $A$)  
  - edges between an activity in the technosphere and an activity in the "biosphere" (an element $b_{kj}$ of the biosphere matrix $B$).

One can iterate through **all** exchanges that have a given activity as `output`

In [ ]:
for exc in random_act.exchanges():
    print(exc)

One can also iterate through subsets of the exchanges:  
  - Technosphere exchanges: exchanges linking to other activities in the technosphere, `activity.technosphere()`  
  - Biosphere exchanges: AKA elementary flows, linking to activities in the biosphere database `activity.biosphere()`  
  - Production exchange: the reference flow of the activity `activity.production`  

Let's assign a **technosphere exchange** to a variable to learn more about it:

In [ ]:
random_techno_exchange = [exc for exc in random_act.technosphere()][0]
random_techno_exchange

In [ ]:
type(random_techno_exchange)

Again, the type is a proxy (refer to the diagram above about the different translation layers).

In [ ]:
# Amount, or weight of the edge
random_techno_exchange.amount

In [ ]:
# Activity the exchange stems from
random_techno_exchange.input

In [ ]:
# Activity the exchange terminates in
random_techno_exchange.output

In [ ]:
# Exchange as a dictionary
random_techno_exchange.as_dict()

Let's now look at a production exchange

In [ ]:
random_prod_exchange = [exc for exc in random_act.production()][0]
random_prod_exchange

We can also check **who consumes** output of this activity.

In [ ]:
list(random_act.consumers())

**Exercise:** Assign a biosphere flow to a variable, and check the following:  
  - Is the output the same as for the technosphere exchange?  
  - From what database does the biosphere exchange come from?  
  - What is the amount of the exchange (i.e. the weight of the edge connecting the two activities)?
  
NOTE: If you get a ` list index out of range` error when trying to subscript your list comprehension, it means your list comprehension is empty, i.e. that there are no biosphere flows associated with the activity.

In [ ]:
# Assign the exchange to a variable:
random_bio_exchange = [exc for exc in random_act.biosphere()][0]
random_bio_exchange

In [ ]:
# Output of biosphere exchange
random_bio_exchange.output

In [ ]:
# Is it the same as the output of the technosphere exchange? It should be!
random_bio_exchange.output == random_techno_exchange.output

In [ ]:
# Database of the random biosphere exchange input - `.input`directly returns the activity proxy!
random_bio_exchange.input.key[0]

In [ ]:
# Amount of exchange
random_bio_exchange['amount']

#### Loaded LCI databases

It is possible to load the entire database into a dictionary.  
This greatly speeds up work if you need to iterate over all activities or exchanges. The resulting object is quite big, so you should do this only if the gain in efficiency is worth it.

In [ ]:
# Let's not do this in the seminar, ok?
eidb_loaded = eidb.load()

## 2) My first LCA

Brightway has a so-called LCA object.  
It is instantiated using `LCA(args)`.  
The only required argument is a functional unit, described by a dictionary with keys = activities and values = amounts (more [here](https://docs.brightwaylca.org/lca.html#specifying-a-functional-unit)).  
A second argument that is often passed is an LCIA method, passed using the method tuple.  

### 2.1) General syntax of LCA calculations

Let's create our first LCA object using our random activity and our IPCC method.  

In [ ]:
ipcc_2021_method.name

In [ ]:
random_act

In [ ]:
myFirstLCA_quick = bw2calc.LCA({random_act:1}, ipcc_2021_method.name)

<div class="alert alert-info">
    Note: we are now using <b>bw2calc</b>.
</div>

In [ ]:
myFirstLCA_quick = bw2calc.LCA({random_act.key:1}, ipcc_2021_method.name)

The steps to get to the impact score are as follows:

In [ ]:
myFirstLCA_quick.lci()    # Builds matrices, solves the system, generates an LCI matrix.
myFirstLCA_quick.lcia()   # Characterization, i.e. the multiplication of the elements 
                          # of the LCI matrix with characterization factors from the chosen method
myFirstLCA_quick.score    # Returns the score, i.e. the sum of the characterized inventory

Let's not take a closer look at the LCA object and its methods/attributes. We'll do this by creating a new LCA object: 

In [ ]:
myFirstLCA = bw2calc.LCA({random_act:1}, ipcc_2021_method.name)

### 2.2) the `demand` attribute

In [ ]:
myFirstLCA.demand

Who/what is that?

In [ ]:
bw2calc.get_activity(14265)

To access the actual activity from the demand, you would do this:

In [ ]:
demanded_act = list(myFirstLCA.demand.keys())[0]
demanded_act

In [ ]:
demanded_act == random_act

But...

In [ ]:
bw2calc.get_activity(demanded_act) == random_act

There are also other attributes that have simply not been built yet, such as the `demand_array` and the `score`. To generate them, we first need to actually build the matrices. This will be done when calling the `.lci()` method.

### 2.3) Reminder of the system that needs to be solved in calculating an LCI

Before actually running the `.lci()` method, here's a quick refresher of the actual calculation that Brightway will need to do to calculate the inventory:  

$g=BA^{-1}f$  

where:  

  - $A$ = the technosphere matrix  
  - $B$ = the biosphere matrix (matrix with elementary flows)  
  - $f$ = the final demand vector  
  - $g$ = the inventory  

**Discussion:** Knowing what you do about the structure of Brightway (notably, activities and exchanges), what needs to happen to generate these matrices?

To consider:  
  - how should the order of the rows and columns be determined?  
  - how should we keep track of what is in each row and column?  
  - The parameters in the matrices are sometimes actually probability distribution functions - how should we consider this uncertainty information?  
  - The matrices are *sparse*, i.e. they are mostly made up of zeros. Should we consider this? Why? How?

### 2.4) Building the matrices

#### Structured arrays

LCI data imported in Brightway is stored in the `databases.db` database, discussed above.  
It is also stored as a [data package](https://specs.frictionlessdata.io/data-package/). 

Let's first call `.lci()` to create all the matrices.

In [ ]:
myFirstLCA.lci()

#### The final demand vector (f):
The demand array is the $f$ in $As=f$. 
It is an attribute of the LCA object:

In [ ]:
myFirstLCA.demand_array

In [ ]:
len(myFirstLCA.demand_array)

In [ ]:
myFirstLCA.demand_array.sum()

In [ ]:
col_index = np.argwhere(myFirstLCA.demand_array).squeeze().item(0)
col_index

Who is that?

#### Dictionaries that map between incides and activities

One of the useful things that the `MatrixBuilder` produces are `dictionaries` that map row and column numbers to the keys of activities.  There are three such dictionaries, all directly accessible as attributes of the LCA object:
 - `activity_dict`: Columns in the technosphere matrix $A$ or biosphere matrix $B$
 - `product_dict` : Rows in the technosphere matrix $A$  
 - `biosphere_dict`: Rows in the biosphere matrix $B$

In [ ]:
list(myFirstLCA.activity_dict.items())[:4]

But we want to know the opposite: which row or column number for a given acivity/product?

As a convenience, Brightway offers a method that will generate the three reverse dictionaries simultaneously.  
`.reverse_dict()` returns three reverse dicts (reverse activity dict,  reverse product dict,  reverse biosphere dict) *in that order*. The syntax for creating and assigning these reverse dicts is therefore: 

In [ ]:
rev_act_dict, rev_prod_dict, rev_bio_dict = myFirstLCA.reverse_dict()

In [ ]:
rev_prod_dict[col_index] == random_act.id

#### $A$ and $B$ Matrices

We can also access the matrices that were constructed. Let's look at the technosphere matrix ($A$).  
The ** $A$ matrix**, with each element $a_{ij}$ provides information on the amount of input or output of product $i$ from activity $j$. When $i=j$, the element $a_{ij}$ is the *reference flow* for the activity described in the column.

In [ ]:
myFirstLCA_quick.technosphere_matrix

I am told that the dimensions of the matrix is $n*n$ where $n$ is the number of activities in my product system, and that the amount of actually stored elements is much less than $n^2$ (because the matrix is *sparse* and zero values are not stored).  

We can have an idea of what it stores by printing it out:

In [ ]:
print(myFirstLCA_quick.technosphere_matrix)

It therefore stores both the coordinates and the values (as expected).
We can slice this matrix using coordinates. For example, let's say we wanted a view of the exchanges associated with the unit process providing our functional unit.  
We already know found the column number for that activity: 

To return the whole column from the $A$ matrix, we therefore slice the $A$ matrix.  
Python notes:  
  - In Python, slicing is done using []
  - We specify rows first, then columns  
  - `:` refers to "the whole row" or "the whole column" (depending if it is passed first or second in the []) 

In [ ]:
myColumn = myFirstLCA.technosphere_matrix[:, col_index]
myColumn

Printing this out gives:

In [ ]:
print(myColumn)

Not too useful: it would be better to get the *names* to these exchanges.  
We need to do two things:  
  - Get the indices from the CSR matrix (we can do this by converting it to a sparse matrix in `COOrdinate` format first)  
  - Get the activity code for the each index (we can do this using the reverse of the `activity_dict`)  
  - Use `get_activity` to access the actual names of the activities.  

1) Converting the CSR matrix to a COO matrix:  

In [ ]:
myColumnCOO = myColumn.tocoo()
myColumnCOO

It is still a sparse matrix with the same number of elements, and it looks quite like the CSR version when we print it out:

In [ ]:
print(myColumnCOO)

However, we can directly access the rows and column indices using `row` and `col`:  

In [ ]:
myColumnCOO.row

2) Get the activity code for each element using the **reverse** product dictionary we produced above:

In [ ]:
# Using a list comprehension:
[rev_prod_dict[i] for i in myColumnCOO.row]

It would be even nicer to get the names for these:

In [ ]:
names_of_my_inputs = [bw2data.get_activity(rev_prod_dict[i])['name'] for i in myColumnCOO.row]
names_of_my_inputs

We can put these in a neat Pandas Series, with actual names and amounts:

In [ ]:
# First create a dict with the information I want:
myColumnAsDict = dict(zip(names_of_my_inputs, myColumnCOO.data))
# Create Pandas Series from dict
pd.Series(myColumnAsDict, name="Nice series with information on exchanges in my foreground process")

Alternative way to generate similar information without even looking at the matrices:

In [ ]:
pd.Series({bw2data.get_activity(exc.input)['name']:exc.amount for exc in random_act.technosphere()}, 
          name="alternative way to generate exchanges")

Note the differences:  
  - The reference flow is not there (activity.technosphere() only returns technoshere exchanges where the input is not equal to the output)  
  - The values are positive, not negative (because the $A$ matrix is $I-Z$ where $Z$ contains the information on these inputs.

**Exercise**: Create a Pandas Series with the elementary flows of the activity supplying the reference flow for myFirstLCA.

In [ ]:
# Solution
myBioColumn = myFirstLCA.biosphere_matrix[:, col_index]
myBioValues = myBioColumn.tocoo().data
myBioNames = [bw2data.get_activity(rev_bio_dict[row])['name'] for row in myBioColumn.tocoo().row]
pd.Series(dict(zip(myBioNames, myBioValues)))

### 2.5) Solution to the inventory calculation

We saw above how `.lci()` produced the $A$ and $B$ matrices.  
`.lci()` also *solves* the equation $As=f$ and calculated the inventory by multiplying the solution to this equation by the biosphere matrix.  

#### Supply array

Vector containing the amount each activity will need to provide to meet the functional demand, i.e. $s=A^{-1}f$.

In [ ]:
myFirstLCA_quick.supply_array

In [ ]:
myFirstLCA_quick.supply_array.shape

**Inventory matrix**  
Contains the inventory *by activity* (i.e. not summed). In other words, we do not have $g=BA^{-1}f$, but rather $G=B \cdot diag(A^{-1}f)$

In [ ]:
myFirstLCA_quick.inventory

We can aggregate the LCI results along the columns (i.e. calculate the cradle-to-gate inventory):

In [ ]:
LCI_cradle_to_gate = myFirstLCA_quick.inventory.sum(axis=1)
LCI_cradle_to_gate.shape

**Exercise:** Get the total (cradle-to-gate) emissions of nitrous oxide emitted to air in the "urban air" subcompartment.

In [ ]:
NOx_act = [act for act in my_bio if 'Dinitrogen monoxide' in act['name']
                       and 'urban air close to ground' in str(act['categories'])
         ][0]
print(NOx_act.key)
print(NOx_act.id)

In [ ]:
NOx_row = myFirstLCA_quick.biosphere_dict[4192]
NOx_row

In [ ]:
myFirstLCA_quick.inventory[NOx_row, :].sum()

### 2.7) LCIA

The LCIA calculation is done via the `.lcia()` method.

In [ ]:
myFirstLCA.lcia()

A number of other matrices are now available:

In [ ]:
# Matrix of characterization factors:
myFirstLCA_quick.characterization_matrix

In [ ]:
myFirstLCA.characterization_matrix.shape

In [ ]:
# Matrix of characterized inventory flows
myFirstLCA_quick.characterized_inventory

The overall score is now an attribute of the LCA object: 

In [ ]:
myFirstLCA.score

We also could have determined what this score was by summing the elements of our `characterized_inventory` matrix:

In [ ]:
myFirstLCA.characterized_inventory.sum()

We could also have calculated it by multiplying the inventory and characterization factors ourselves:

In [ ]:
(myFirstLCA.characterization_matrix * myFirstLCA.inventory).sum()

We could also calculate the score by elementary flow (summing columns for each rows), irrespective of the unit process that produced it:

In [ ]:
#Axis is the dimension I want to sum over
elementary_flow_contribution = myFirstLCA.characterized_inventory.sum(axis=1) 

In [ ]:
elementary_flow_contribution.shape

Notice that is has **two** dimensions. The result is in fact a one-dimensional matrix:

In [ ]:
type(elementary_flow_contribution)

To convert it to an array (probably more useful for many purposes), you can use any of the following approaches:

In [ ]:
elementary_flow_contribution.A1 
#np.squeeze(np.asarray(elementary_flow_contribution))
#np.asarray(elementary_flow_contribution).reshape(-1)
#np.array(elementary_flow_contribution).flatten()
#np.array(elementary_flow_contribution).ravel()

**Exercise:** Create a Pandas series that has the scores per unit process, sorted by value (contribution analysis)

In [ ]:
# Create array with the results per column (i.e. per activity)
results_by_activity = (myFirstLCA_quick.characterized_inventory.sum(axis=0)).A1

In [ ]:
# Create a list of names in columns
list_of_names_in_columns = [
    bw2data.get_activity(rev_act_dict[col])['name'] 
    for col in range(myFirstLCA.characterized_inventory.shape[1])
]

In [ ]:
pd.Series(index=list_of_names_in_columns, data=results_by_activity).sort_values(ascending=False).head(10)

## 3) My second LCA: Comparative LCA

Let's choose two activities to compare, say Swiss electricity produced from respectively a run-of-river hydropower plant and a wind turbine.

**Exercise**: assign the two activities to variables `hydro` and `wind` respectively.

In [ ]:
wind = [act for act in eidb if "wind" in act['name']
        and "<1MW" in act['name'] 
        and "kilowatt hour" in act['unit']
        and act["location"] == "DK"
       ][0]
wind

In [ ]:
hydro = [act for act in eidb if "hydro" in act['name'] 
                     and "river" in act['name'] 
                     and "CH" in act['location']
                     and "kilowatt hour" in act['unit']
                     ][0]
hydro

Let's also compare these according to their carbon footprint as measured with the IPCC method we already selected above:

In [ ]:
ipcc_2021_method

#### One-at-a-time approach:

In [ ]:
hydroLCA = bw2calc.LCA({hydro:1}, ipcc_2021_method.name)
hydroLCA.lci()
hydroLCA.lcia()
hydroLCA.score

**Exercise:** Do the LCA for `wind`:

In [ ]:
windLCA = bw2calc.LCA({wind:1}, ipcc_2021_method.name)
windLCA.lci()
windLCA.lcia()
windLCA.score

In [ ]:
#Compare results:
if windLCA.score > hydroLCA.score:
    print("Hydro is preferable")
elif windLCA.score < hydroLCA.score:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

Do one "delta" LCA:

In [ ]:
deltaLCA = bw2calc.LCA({wind:1, hydro:-1}, ipcc_2021_method.name)
deltaLCA.lci()
deltaLCA.lcia()
deltaLCA.score

In [ ]:
if deltaLCA.score>0:
    print("Hydro is preferable")
elif deltaLCA.score<0:
    print("Wind is preferable")
else:
    print("Both options have the same climate change indicator result")

## 4) My third LCA - Multiple impact categories

Say we want to evaluate the indicator results for our randomAct for all EF v3 categories (with long-term emissions).

In [ ]:
# Make a list of all impact method names (tuples):
EFV3 = [method for method in bw2data.methods if "EF v3.1" in str(method) 
        and "no LT" not in str(method)
        and "EN15804" not in str(method)][:8]
EFV3

Simplest way: for loop, using `switch method`

In [ ]:
def get_lcia_scores(products, categories, results):
    lca = bw2calc.LCA({products[0]: 1}, categories[0])
    lca.lci()
    lca.lcia()
    
    method_matrices = [lca.characterization_matrix.copy()]
    
    for other_method in categories[1:]:
        # Only build each characterization matrix once instead of once per product
        lca.switch_method(other_method)
        method_matrices.append(lca.characterization_matrix.copy())
    
    for i, product in enumerate(products):
        lca.lci({product.id: 1})
        for j, characterization_matrix in enumerate(method_matrices):
            results[i, j] = (characterization_matrix * lca.inventory).sum()
    
    return results

results = np.zeros((2, len(EFV3)))
results = get_lcia_scores([hydro, wind], EFV3, results)
results

In [ ]:
df = pd.DataFrame(columns=[e[1] for e in EFV3], index=[hydro['name'][:32], wind['name'][:30], ], data=results)
df

In [ ]:
df.div(df.iloc[0], 1).plot(kind="bar")

## Uncertainty

Values of exchanges in datasets may come with uncertainty.
See [documentation](https://stats-arrays.readthedocs.io/en/latest/) of `stats_array`.

Let's find out how uncertainty is defined in ecoinvent:

In [ ]:
act = eidb.random()
for e in act.exchanges():
    if e["type"] == "technosphere":
        pprint(e.as_dict())
        print(e.uncertainty_type)
        print(e.uncertainty)
        break

### Running a stochastic analysis (Monte Carlo)

In [ ]:
iterations = 10
results = np.zeros((2, iterations))

In [ ]:
EFV3[1]

In [ ]:
for a, activity in enumerate([hydro, wind]):
    lca = bw2calc.LCA(
        demand={activity: 1},
        method=EFV3[1],
        use_distributions=True,
    )
    lca.lci()
    lca.lcia()
    
    results[a] = [lca.score for _ in zip(range(iterations), lca)]

Let's visualize the results:

In [ ]:
df = pd.DataFrame(results.T, columns=["Wind", "Hydro"])
df

In [ ]:
df[df.sum(axis=1)>0].boxplot()

That's it for the time being.

Additional resources to dig the topic deeper:
* Exercises on [bw2calc and bw_processing](https://github.com/Depart-de-Sentier/Spring-School-2024/blob/main/class-materials/brightway-basics/2%20-%20Building%20and%20using%20matrices%20in%20bw2calc.ipynb)